In [1]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [2]:
list=[['user1',   '12/1/19 8:00'],
      ['user1' ,  '12/1/19 10:00'],
      ['user1'  , '12/1/19 23:00'],
      ['user1'   ,'12/2/19 7:00'],
      ['user1'   ,'12/2/19 8:00'],
      ['user1'   ,'12/2/19 10:00'],
      ['user1'   ,'12/3/19 23:00'],
      ['user1'   ,'12/4/19 7:00'],
      ['user2'   ,'12/4/19 8:00'],
      ['user2'   ,'12/5/19 5:00'],
      ['user2'   ,'12/6/19 0:00']]
df=spark.createDataFrame(list,['user','login'])
df.show()

+-----+-------------+
 user| login|
+-----+-------------+
user1| 12/1/19 8:00|
user1|12/1/19 10:00|
user1|12/1/19 23:00|
user1| 12/2/19 7:00|
user1| 12/2/19 8:00|
user1|12/2/19 10:00|
user1|12/3/19 23:00|
user1| 12/4/19 7:00|
user2| 12/4/19 8:00|
user2| 12/5/19 5:00|
user2| 12/6/19 0:00|
+-----+-------------+

In [3]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

w=Window().partitionBy("user").orderBy("login").rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
w2=Window().partitionBy("user", "index").orderBy("login")
df1=df.withColumn("login", F.to_timestamp("login","MM/dd/yy HH:mm"))
df1.withColumn("firstlogin", F.first(F.col("login")).over(w))\
  .withColumn("index", F.when(((unix_timestamp(col("login")) - unix_timestamp(col("firstLogin"))) / 86400) == 1, F.lit(0)).otherwise(((unix_timestamp(col("login")) - unix_timestamp(col("firstLogin"))) / 86400).cast("int")))\
  .withColumn("Duplicate", when(row_number().over(w2) == 1, lit("N")).otherwise(lit("Y"))) \
   .orderBy("user", "login") \
   .show()



+-----+-------------------+-------------------+-----+---------+
 user| login| firstlogin|index|Duplicate|
+-----+-------------------+-------------------+-----+---------+
user1|2019-12-01 08:00:00|2019-12-01 08:00:00| 0| N|
user1|2019-12-01 10:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-01 23:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-02 07:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-02 08:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-02 10:00:00|2019-12-01 08:00:00| 1| N|
user1|2019-12-03 23:00:00|2019-12-01 08:00:00| 2| N|
user1|2019-12-04 07:00:00|2019-12-01 08:00:00| 2| Y|
user2|2019-12-04 08:00:00|2019-12-04 08:00:00| 0| N|
user2|2019-12-05 05:00:00|2019-12-04 08:00:00| 0| Y|
user2|2019-12-06 00:00:00|2019-12-04 08:00:00| 1| N|
+-----+-------------------+-------------------+-----+---------+

In [4]:
((unix_timestamp(col("login")) - unix_timestamp(col("firstLogin"))) / 86400).cast("int"))

In [5]:
from pyspark.sql.functions import col, lag, unix_timestamp, to_timestamp, lit, when, row_number, first
from pyspark.sql import Window

w = Window.partitionBy("user", "index").orderBy("login")
df2 = df.withColumn("login", to_timestamp(col("login"), "MM/dd/yy HH:mm"))

df2.join(df2.groupBy("user").agg(first("login").alias("firstLogin")), "user", "left") \
   .withColumn("index", F.when(((unix_timestamp(col("login")) - unix_timestamp(col("firstLogin"))) / 86400) == 1, F.lit(0)).otherwise(((unix_timestamp(col("login")) - unix_timestamp(col("firstLogin"))) / 86400).cast("int"))) \
   .withColumn("Duplicate", when(row_number().over(w) == 1, lit("N")).otherwise(lit("Y"))) \
   .orderBy("user", "login") \
   .show(20)

+-----+-------------------+-------------------+-----+---------+
 user| login| firstLogin|index|Duplicate|
+-----+-------------------+-------------------+-----+---------+
user1|2019-12-01 08:00:00|2019-12-01 08:00:00| 0| N|
user1|2019-12-01 10:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-01 23:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-02 07:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-02 08:00:00|2019-12-01 08:00:00| 0| Y|
user1|2019-12-02 10:00:00|2019-12-01 08:00:00| 1| N|
user1|2019-12-03 23:00:00|2019-12-01 08:00:00| 2| N|
user1|2019-12-04 07:00:00|2019-12-01 08:00:00| 2| Y|
user2|2019-12-04 08:00:00|2019-12-04 08:00:00| 0| N|
user2|2019-12-05 05:00:00|2019-12-04 08:00:00| 0| Y|
user2|2019-12-06 00:00:00|2019-12-04 08:00:00| 1| N|
+-----+-------------------+-------------------+-----+---------+

In [6]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

In [7]:
list=[[ 'a1' ,     'b1'  ,    'c1'   ,   'd1'],
     ['a1'  ,   'b2'    ,  'c1'      ,'d1'],
     ['a1'   ,   'b3'   ,   'c1'     , 'd1'],
     ['a1'  ,    'b4'   ,   'c1'     , 'd2'],
     ['a1'   ,   'b5'    ,  'c2'     , 'd2'],
     ['a1'   ,   'b6'    ,  'c2'     , 'd2'],
     ['a1'  ,    'b7'    ,  'c1'     , 'd3'],
     ['a1'  ,    'b8'    , 'c2'      ,'d3'],
     ['a1'  ,    'b9'    ,  'c3'     , 'd3'],
     ['a1'  ,    'b10'   ,  'c1'     , 'd2'],
     [ 'a1'  ,   'b11'   ,  'c2'     , 'd3'],
     ['a2'  ,    'b12'   ,  'c1'     , 'd1'],
     ['a3'   ,   'b13'   ,  'c1'     , 'd1']]

df=spark.createDataFrame(list,['col_1','col_2','col_3','col_4'])

df.show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 a1| b1| c1| d1|
 a1| b2| c1| d1|
 a1| b3| c1| d1|
 a1| b4| c1| d2|
 a1| b5| c2| d2|
 a1| b6| c2| d2|
 a1| b7| c1| d3|
 a1| b8| c2| d3|
 a1| b9| c3| d3|
 a1| b10| c1| d2|
 a1| b11| c2| d3|
 a2| b12| c1| d1|
 a3| b13| c1| d1|
+-----+-----+-----+-----+

In [8]:
w=Window().partitionBy("col_3","col_4")
w2=Window().partitionBy()

df.withColumn("count", F.count("*").over(w)).withColumn("max", F.max("count").over(w2)).filter("count=max")\
  .groupBy(F.col("col_1")).agg(*(F.first(x).alias(x) for x in df.columns if x!= 'col_1' and x!='count')).show()

+-----+-----+-----+-----+
col_1|col_2|col_3|col_4|
+-----+-----+-----+-----+
 a1| b1| c1| d1|
 a2| b12| c1| d1|
 a3| b13| c1| d1|
+-----+-----+-----+-----+

In [9]:
w=Window().partitionBy("col_3","col_4")


df.withColumn("count", F.count("*").over(w))\
  .groupBy(F.col("col_1")).agg(F.max("count"),*(F.first(x).alias(x) for x in df.columns if x!= 'col_1' and x!='count')).show()

+-----+----------+-----+-----+-----+
col_1|max(count)|col_2|col_3|col_4|
+-----+----------+-----+-----+-----+
 a3| 5| b13| c1| d1|
 a2| 5| b12| c1| d1|
 a1| 5| b7| c1| d3|
+-----+----------+-----+-----+-----+

In [10]:
.groupBy(F.col("col_1")).agg(F.first(*(x for x in df.columns if x!=col_1))).show()

In [11]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.storagelevel import StorageLevel

In [12]:
list1= [[1,'Michael' ,'Jackson', 'mj@yahoo.com'],
       [2,'Roger','Moore','rm@rocketmail.com'],
       [3,'Angela','Merkel','am@dw.de']]




list2= [[1,'Michael' ,'Jordan', 'mj@yahoo.com'],
       [2,'Gordon','Moore','rm@rocketmail.com'],
       [3,'Angela','Merkle','am@dw.com']]


df1= spark.createDataFrame(list1,['id'  ,'fname',   'lname',   'email'])
df2= spark.createDataFrame(list2,['id'  ,'fname','lname' ,'email'])

df1.show()
df2.show()

+---+-------+-------+-----------------+
 id| fname| lname| email|
+---+-------+-------+-----------------+
 1|Michael|Jackson| mj@yahoo.com|
 2| Roger| Moore|rm@rocketmail.com|
 3| Angela| Merkel| am@dw.de|
+---+-------+-------+-----------------+

+---+-------+------+-----------------+
 id| fname| lname| email|
+---+-------+------+-----------------+
 1|Michael|Jordan| mj@yahoo.com|
 2| Gordon| Moore|rm@rocketmail.com|
 3| Angela|Merkle| am@dw.com|
+---+-------+------+-----------------+

In [13]:
df3=df1.join(df2.select(F.col("id"),F.col("fname").alias("fname1"),F.col("lname").alias("lname1"),F.col("email").alias("email1")),['id'])
collected=df3.withColumn("fname1", F.when(F.col("fname1")!=F.col("fname"),F.lit("fname")).otherwise(F.lit(None)))\
  .withColumn("lname1", F.when(F.col("lname1")!=F.col("lname"),F.lit("lname")).otherwise(F.lit(None)))\
  .withColumn("email1", F.when(F.col("email1")!=F.col("email"),F.lit("email")).otherwise(F.lit(None)))\
  .withColumn("different", F.array(*(x for x in ['fname1','lname1','email1']))).select("id","different")\
  .withColumn("different", F.expr("""filter(different, x-> x is  not null)""")).orderBy("id").collect()
dic={}
for i in collected:
  dic.update({i[0] : i[1]})
dic

Out[48]: {1: ['lname'], 2: ['fname'], 3: ['lname', 'email']}

In [14]:
df3=df1.join(df2.select(F.col("id"),F.col("fname").alias("fname1"),F.col("lname").alias("lname1"),F.col("email").alias("email1")),['id'])
collected=df3.withColumn("fname1", F.when(F.col("fname1")!=F.col("fname"),F.lit("fname")).otherwise(F.lit(None)))\
  .withColumn("lname1", F.when(F.col("lname1")!=F.col("lname"),F.lit("lname")).otherwise(F.lit(None)))\
  .withColumn("email1", F.when(F.col("email1")!=F.col("email"),F.lit("email")).otherwise(F.lit(None)))\
  .withColumn("different", F.array(*(x for x in ['fname1','lname1','email1']))).select("id","different")\
  .withColumn("different", F.expr("""filter(different, x-> x is  not null)""")).orderBy("id").toJSON().collect()

In [15]:
collected

Out[50]: ['{"id":1,"different":["lname"]}',
 '{"id":2,"different":["fname"]}',
 '{"id":3,"different":["lname","email"]}']

In [16]:
dic={}
for i in collected:
  dic.update({i[0] : i[1]})

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-3186152684721686> in <module> 
 1 dict = { } 
 ----> 2 dic . update ( ( { i [ 0 ] : i [ 1 ] } ) for i in collected ) 

 ValueError : dictionary update sequence element #0 has length 1; 2 is required

In [18]:
dic

Out[39]: {1: ['lname'], 2: ['fname'], 3: ['lname', 'email']}